# Reinforcement learning

In [ ]:
import gymnasium as gym
import numpy as np
# for allowing abstract methodes (closest thing to interface)
from abc import ABC, abstractmethod
from IPython.display import display, clear_output

class Agent(ABC):

    @abstractmethod
    def _init_weights(self):
        pass

    @abstractmethod
    def copy(self):
        pass

    @abstractmethod
    def get_action(self, observation=None):
        pass

    @abstractmethod
    def mutate(parents=None, mutation_rate=None):
        pass

# simulate single training run
def simulate_env(env, agent):

    # Schrijf dit snel
    
    pass

# train a reinforcment learning agent
def train_agent(env, agent, population_size = 50, mutation_rate=0.4, num_generations = 100, num_episodes=5):

    # Initialize the population
    population = [agent.copy() for _ in range(population_size)]

    # number of generations in the algorithm
    for generation in range(num_generations):
        # Evaluate each individual in the population
        scores = []
        
        # Select the top-performing individuals
    
        # Create a new population by mutating and recombining the elite individuals
    
        # Print the best score in this generation
        best_score = max(scores)
        print(f"Generation {generation + 1}: Best Score = {best_score}")
    
    # return best individual
    return best_individual


## Evolutionary algorithms


In [ ]:
class EvolutionaryAgent(Agent):
    num_parents = 1

    def __init__(self):
        super().__init__()

        # vul de constructor indien nodig aan
        
        self._init_weights()
        
    def _init_weights(self):
        # maak een neuraal netwerk/de gewichten ervan
        pass
                
    def copy(self):
        # maak een nieuwe agent van deze klasse

        pass
        

    def get_action(self, observation=None):
        # zoek de beste actie
        pass

    def mutate(self, parents=None, mutation_rate=None):
        # return een nieuw kind op basis van de gewichten van deze agent
        pass

In [ ]:
# Define the MountainCar environment
env = gym.make("MountainCar-v0")

# Hyperparameters
population_size = 100
mutation_rate = 0.3
num_generations = 100
num_episodes = 5

# RL agent with internally a NN with a hidden layer of 8 neurons
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
agent = EvolutionaryAgent(num_inputs=input_size, num_outputs=output_size, hidden_layer_sizes=[8])

best_evolutionary_agent = train_agent(env, agent, population_size=population_size, mutation_rate=mutation_rate, num_generations=num_generations, num_episodes=num_episodes)

In [ ]:
# Evaluate the best individual
env = gym.make("MountainCar-v0", render_mode="human")

for episode in range(5):
    score = simulate_env(env, best_evolutionary_agent)
    print(f"Best Individual Score: {score}")

In [ ]:
import matplotlib.pyplot as plt

def plot_action_space(agent):

    results = []
    xs = np.arange(-1.2, 0.6, 0.05)
    ys = np.arange(-0.07, 0.07, 0.001)

    for x in xs:
        tmp = []
        for y in ys:
            tmp.append(agent.get_action(np.array([x, y])))
        results.append(tmp)
    results = np.array(results)

    plt.imshow(results, cmap='gray', interpolation='none') 

plot_action_space(best_evolutionary_agent)